In [3]:
import torch
from torchvision.models import convnext_base, ConvNeXt_Base_Weights
from torchvision.models.feature_extraction import get_graph_node_names
from torchvision.models.feature_extraction import create_feature_extractor
from tqdm import tqdm
from torch.utils.data.distributed import DistributedSampler
from torch.utils.data import DataLoader, SequentialSampler


import pickle
import os

import faiss
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize

import sys
sys.path.append("..")
from src.image_data import ImageClassificationDataset

### Load ConvNeXt Model

In [ ]:
DATA_PATH = '/mnt/ssd/ronak/datasets/cifar10'
DEVICE = 'cuda:0'

In [3]:
model = convnext_base(weights=ConvNeXt_Base_Weights.IMAGENET1K_V1).to(DEVICE)
train_nodes, eval_nodes = get_graph_node_names(model)

/home/ronak/miniconda3/envs/dl/lib/python3.11/site-packages/torch/overrides.py:110: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  torch.has_cuda,
/home/ronak/miniconda3/envs/dl/lib/python3.11/site-packages/torch/overrides.py:111: UserWarning: 'has_cudnn' is deprecated, please use 'torch.backends.cudnn.is_available()'
  torch.has_cudnn,
/home/ronak/miniconda3/envs/dl/lib/python3.11/site-packages/torch/overrides.py:117: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  torch.has_mps,
/home/ronak/miniconda3/envs/dl/lib/python3.11/site-packages/torch/overrides.py:118: UserWarning: 'has_mkldnn' is deprecated, please use 'torch.backends.mkldnn.is_available()'
  torch.has_mkldnn,


In [4]:
return_nodes = {
    # node_name: user-specified key for output dict
    'avgpool': 'features',
}
body = create_feature_extractor(model, return_nodes=return_nodes)

In [5]:
root = DATA_PATH
x_train = np.transpose(np.load(os.path.join(root, "x_train.npy")), axes=[0, 3, 1, 2])
y_train = np.load(os.path.join(root, "y_train.npy"))
print(x_train.shape)

batch_size = 256
transforms = ConvNeXt_Base_Weights.IMAGENET1K_V1.transforms()
train_dataset = ImageClassificationDataset(x_train, y_train, transforms)
dataloader = DataLoader(
    train_dataset, sampler=SequentialSampler(train_dataset), batch_size=batch_size
)

(50000, 3, 32, 32)


In [6]:
all_image_features, all_labels, all_idx = [], [], []
with torch.no_grad():
    for i, batch in tqdm(enumerate(dataloader)):
        idx, images, labels = batch
        image_features = body(images.to(DEVICE))['features'].squeeze()
        image_features /= image_features.norm(dim=-1, keepdim=True)
        all_image_features.append(image_features)
        all_labels.append(labels)
        all_idx.append(idx)
        
all_image_features = torch.cat(all_image_features).cpu().detach().numpy()
all_labels = torch.cat(all_labels).cpu().detach().numpy()
all_idx = torch.cat(all_idx).cpu().detach().numpy()

0it [00:00, ?it/s]/home/ronak/miniconda3/envs/dl/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
196it [06:56,  2.12s/it]


In [7]:
torch.save(all_image_features, os.path.join(DATA_PATH, "convnext_base_features.pt"))
torch.save(all_labels, os.path.join(DATA_PATH, "convnext_base_labels.pt"))
torch.save(all_idx, os.path.join(DATA_PATH, "convnext_base_idx.pt"))

## Perform Quantization

In [31]:
NUM_CLUSTERS = 25
SEED = 11182023
DATA_PATH = '/mnt/ssd/ronak/datasets/cifar10'

In [32]:
all_image_features = torch.load(os.path.join(DATA_PATH, "convnext_base_features.pt"))
all_labels = torch.load(os.path.join(DATA_PATH, "convnext_base_labels.pt"))
all_idx = torch.load(os.path.join(DATA_PATH, "convnext_base_idx.pt"))

In [33]:
class KMeans(object):
    
    def __init__(self, norm, pca, idx, index, marginal):
        self.norm = norm
        self.pca = pca
        self.idx = idx
        self.index = index
        self.marginal = marginal
    
    def clustering(self, features):
        data = features
        if self.norm in ['l2', 'l1']:
            data = normalize(data, norm=self.norm, axis=1)
        data = self.pca.transform(data)[:, :self.idx+1]
        data = data.astype(np.float32)
        _, labels = self.index.search(data, 1)
        labels = labels.reshape(-1)
        return labels


def cluster_feat(features, num_clusters,
                 norm='none', whiten=True,
                 pca_max_data=-1,
                 explained_variance=0.9,
                 num_redo=5, max_iter=500, seed=0):
    assert 0 < explained_variance < 1
    assert norm in ['none', 'l2', 'l1', None]
    data1 = features
    if norm in ['l2', 'l1']:
        data1 = normalize(data1, norm=norm, axis=1)
    pca = PCA(n_components=None, whiten=whiten, random_state=seed+1)
    if pca_max_data < 0 or pca_max_data >= data1.shape[0]:
        pca.fit(data1)
    elif 0 < pca_max_data < data1.shape[0]:
        rng = np.random.RandomState(seed+5)
        idxs = rng.choice(data1.shape[0], size=pca_max_data, replace=False)
        pca.fit(data1[idxs])
    else:
        raise ValueError(f'Invalid argument pca_max_data={pca_max_data} with {data1.shape[0]} datapoints')
    s = np.cumsum(pca.explained_variance_ratio_)
    idx = np.argmax(s >= explained_variance)  # last index to consider
    data1 = pca.transform(data1)[:, :idx+1]
    # Cluster
    data1 = data1.astype(np.float32)
    kmeans = faiss.Kmeans(data1.shape[1], num_clusters, niter=max_iter,
                          nredo=num_redo, update_index=True, seed=seed+2, min_points_per_centroid=50)
    kmeans.train(data1)
    index = kmeans.index
    _, labels = index.search(data1, 1)
    
    # Drop clusters with low frequency
    ids, counts = np.unique(labels, return_counts=True)
    to_remove = ids[counts < 50]
    if len(to_remove) > 0:
        index.remove_ids(to_remove)
        _, labels = index.search(data1, 1)
    
    _, counts = np.unique(labels, return_counts=True)
    cluster = KMeans(norm, pca, idx, index, counts/np.sum(counts))
    return labels.reshape(-1), cluster

In [34]:
image_labels, image_cluster = cluster_feat(all_image_features, NUM_CLUSTERS, seed=SEED)

label_to_idx = np.argsort(all_idx)
print(all_idx[label_to_idx])

# have the labels correspond to the indices in order.
image_labels_sorted = image_labels[label_to_idx]
class_labels_sorted = all_labels[label_to_idx]

print(image_labels_sorted.shape)
print(class_labels_sorted.shape)

[    0     1     2 ... 49997 49998 49999]
(50000,)
(50000,)


In [35]:
model_name = "convnext_base"
dataset = "cifar10"
save_dir = f'/mnt/ssd/ronak/datasets/{dataset}/quantization/{model_name}_kmeans_{NUM_CLUSTERS}'

os.makedirs(save_dir, exist_ok=True)

np.save(os.path.join(save_dir, f'image_labels.npy'), image_labels_sorted)
np.save(os.path.join(save_dir, f'class_labels.npy'), class_labels_sorted)

_, counts = np.unique(all_labels, return_counts=True)
y_marginal = counts/np.sum(counts)
x_marginal = image_cluster.marginal

np.save(os.path.join(save_dir, f'image_marginal.npy'), x_marginal)
np.save(os.path.join(save_dir, f'class_marginal.npy'), y_marginal)